# Importação dos dados municipais - 16/12/2020

Os atributos dos vértices do grafo consistem, inicialmente, em:
- população absoluta;
- número de leitos hospitalares;
- coordenadas x,y (graus)
- área territorial (m²);
- série temporal de casos;
- série temporal de óbitos;

a partir dos quais será feita a previsão através de uma rede neural aplicada ao modelo de grafo. Dados derivados, como população na base de 100 mil habitantes, densidade demográfica, casos e mortes por 100 mil habitantes, etc, não serão incluídos no arquivo final.

In [1]:
import numpy as np
import pandas as pd

In [2]:
# pastas para leitura dos dados
path1 = 'dados/fonte/Brasil.io/'
path3 = 'dados/fonte/IBGE_GeoHub/'
path2 = 'dados/gerado/'

Abertura da planilha de casos de contaminação e mortes por dia e município. É importante destacar que há duas datas importantes, a data de coleta (*date*) e a data de registro (*last_available_date*): a primeira se refere a quando que o dado foi obtido do órgão oficial, enquanto a segunda diz a que dia aquele dado se refere.

Isso se dá visto que há dias em que a base de dados oficial não registrou nenhuma mudança, então o dado coletado naquele dia se refere há uma data anterior. Para esses casos, será observado que `data_registro` < `data_coleta` e `repetido` = **True**, além dos valores diários de novos casos e novas mortes estarem zerados.

In [3]:
# série de casos por município
df_c = pd.read_csv(path1 + 'caso_full.csv')
df_c = df_c[ df_c['place_type']=='city' ]
df_c = df_c[ ~df_c.isna()['city_ibge_code'] ]
df_c.drop(columns=['is_last', 'place_type', 'estimated_population_2019'], inplace=True)
df_c.drop_duplicates(inplace=True)
df_c.rename(columns={'city': 'municipio', # nome do município
                     'state': 'estado', # unidade federativa
                     'date': 'data_coleta', # data em que os dados foram obtidos dos órgãos oficiais
                     'is_repeated': 'repetido', # se aquele dado é repetido de algum dia anterior
                     'order_for_place': 'ordem_local', # número sequencial do dia epidemiológico do município
                     'city_ibge_code': 'geocodigo', # código do município de acordo com a base de dados do IBGE
                     'epidemiological_week': 'sem_epidem', # semana epidemiológica
                     'estimated_population': 'populacao', # população estimada de 2020
                     'last_available_confirmed': 'num_casos', # número acumulado de casos
                     'last_available_confirmed_per_100k_inhabitants': 'num_casos_100k', # casos acumulados p/ 100k
                     'last_available_date': 'data_registro', # data a qual os dados coletados se referem
                     'last_available_deaths': 'num_mortes', # número acumulado de mortes
                     'last_available_death_rate': 'taxa_mortes', # taxa de mortes no município
                     'new_confirmed': 'novos_casos', # casos registrados no dia
                     'new_deaths': 'novas_mortes'}, inplace=True) # mortes registradas no dia
df_c = df_c.astype({'populacao': 'int32', 
                    'geocodigo': 'int32',
                    'sem_epidem': 'int16', 
                    'num_casos': 'int32', 
                    'num_mortes': 'int32', 
                    'novos_casos': 'int16', 
                    'novas_mortes': 'int16'}, copy=False)
df_c.sort_values(by=['geocodigo', 'data_coleta'], inplace=True)
df_c

,municipio,geocodigo,data_coleta,sem_epidem,populacao,repetido,num_casos,num_casos_100k,data_registro,taxa_mortes,num_mortes,ordem_local,estado,novos_casos,novas_mortes
46038,Alta Floresta D'Oeste,1100015,2020-05-02,18,22728,False,1,4.39986,2020-05-02,0.0000,0,1,RO,1,0
48424,Alta Floresta D'Oeste,1100015,2020-05-03,19,22728,False,1,4.39986,2020-05-03,0.0000,0,2,RO,0,0
50862,Alta Floresta D'Oeste,1100015,2020-05-04,19,22728,False,1,4.39986,2020-05-04,0.0000,0,3,RO,0,0
53364,Alta Floresta D'Oeste,1100015,2020-05-05,19,22728,False,1,4.39986,2020-05-05,0.0000,0,4,RO,0,0
55935,Alta Floresta D'Oeste,1100015,2020-05-06,19,22728,False,1,4.39986,2020-05-06,0.0000,0,5,RO,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1433958,Brasília,5300108,2021-01-21,3,3055149,False,233973,7658.31716,2021-01-21,0.0174,4067,321,DF,749,7
1439574,Brasília,5300108,2021-01-22,3,3055149,False,234845,7686.85914,2021-01-22,0.0173,4074,322,DF,872,7
1445190,Brasília,5300108,2021-01-23,3,3055149,False,235677,7714.09185,2021-01-23,0.0173,4082,323,DF,832,8
1450806,Brasília,5300108,2021-01-24,4,3055149,False,236430,7738.73877,2021-01-24,0.0173,4090,324,DF,753,8


Abertura da lista de municípios para completar a tabela com as informações estáticas (como população e nome do estado) e retirar esses valores da tabela de casos.

In [4]:
# lista de municipios
df_m = pd.read_csv(path2 + 'lista_mun.csv')
df_m.drop(columns=['geometriaa', 'nome'], inplace=True)
df_m.sort_values(by=['geocodigo'], inplace=True)

# união das colunas com dados de população e estado
df_t = df_c[['geocodigo', 'municipio', 'populacao', 'estado']] # dataframe temporário
df_t = df_t.drop_duplicates().set_index('geocodigo')
df_m = df_m.join(df_t, on='geocodigo')
df_m = df_m.reindex(columns=['geocodigo', 'municipio', 'estado', 'populacao', 'xcoord', 'ycoord'])

# união das colunas com dados de área territorial
df_t = pd.read_csv(path2 + 'lista_geo.csv') # dataframe temporário
df_t.rename(columns={'perimeter': 'perimetro'}, inplace=True)
df_t['area'] = df_t['area'] / (1e3)**2 # mudança de m² para km²
df_t['perimetro'] = df_t['perimetro'] / (1e3) # mudança de m para km
df_t = df_t.set_index('geocodigo')
df_m = df_m.join(df_t, on='geocodigo')

# união das colunas com dados de leitos de uti disponíveis
df_t = pd.read_json(path3 + 'leitos_hosp.json', orient='records') # dataframe temporário
df_t.drop(columns=['leitos_hosp_sus'], inplace=True)
df_t.rename(columns={'leitos_hosp_total': 'leitos_hosp'}, inplace=True)
df_t = df_t.set_index('geocodigo')
df_m = df_m.join(df_t, on='geocodigo')

# exportação e exibição
df_m.to_csv(path2 + 'atrib_estat.csv', index=False)
df_m

,geocodigo,municipio,estado,populacao,xcoord,ycoord,area,perimetro,leitos_hosp
1537,1100015,Alta Floresta D'Oeste,RO,22728,-61.996482,-11.929466,7074.547776,653.606983,31
1542,1100023,Ariquemes,RO,109523,-63.033269,-9.908463,4390.850918,435.427786,142
1538,1100031,Cabixi,RO,5188,-60.544666,-13.495098,1308.666714,196.977604,4
1549,1100049,Cacoal,RO,85893,-61.442944,-11.433865,3795.180484,320.034289,257
1539,1100056,Cerejeiras,RO,16204,-60.818653,-13.194416,2785.079094,382.335889,25
...,...,...,...,...,...,...,...,...,...
5163,5222005,Vianópolis,GO,13977,-48.506671,-16.745736,952.369228,192.584851,28
1968,5222054,Vicentinópolis,GO,8873,-49.807468,-17.730922,734.280378,193.684110,9
4698,5222203,Vila Boa,GO,6312,-47.052931,-15.034124,1064.486720,250.663737,7
5346,5222302,Vila Propício,GO,5882,-48.884897,-15.456882,2186.546234,334.645490,0


In [5]:
# remoção de atributos derivados e campos de informação redundante ou desnecessária
df_c = df_c.reindex(columns=['geocodigo', 'data_coleta', 'novos_casos', 'novas_mortes'])

# preenchimento de datas faltantes
df_t = pd.DataFrame(df_c['data_coleta'].drop_duplicates().sort_values()) # dataframe de série temporal
df_t['dia'] = range(1, len(df_c['data_coleta'].unique()) + 1) # enumeração das datas
df_t['geocodigo'] = 0

df_w = df_c # cópia de trabalho
df_c = pd.DataFrame(columns=df_w.columns)

for cod in df_m['geocodigo']:
    df_t['geocodigo'] = cod
    mask = df_w['geocodigo'] == cod
    merged = df_w[mask].merge(df_t, on=['geocodigo','data_coleta'], how='outer')
    merged.sort_values(by=['data_coleta'], inplace=True)
    df_c = df_c.append(merged, sort=False)

In [6]:
# preenchimento de dados faltantes e ajustes finais
df_c = df_c.fillna(0)
df_c.drop(columns=['data_coleta'], inplace=True)
df_c.rename(columns={'novos_casos':'casos', 'novas_mortes':'mortes'}, inplace=True)
df_c = df_c.astype({'dia':'int16', 'casos':'int16', 'mortes':'int16'})
df_c = df_c.reindex(columns=['geocodigo', 'dia', 'casos', 'mortes'])

df_c.to_csv(path2 + 'atrib_dinam.csv', index=False)
df_c

,geocodigo,dia,casos,mortes
269,1100015,1,0,0
270,1100015,2,0,0
271,1100015,3,0,0
272,1100015,4,0,0
273,1100015,5,0,0
...,...,...,...,...
320,5300108,332,749,7
321,5300108,333,872,7
322,5300108,334,832,8
323,5300108,335,753,8


In [7]:
# exibição dos dados agregados para conferência final
# (qt. de cidades e dias, acumulado de casos e mortes)
df_c.groupby('geocodigo').agg({'dia':'count', 'casos':'sum', 'mortes':'sum'}).astype(int)

,dia,casos,mortes
geocodigo,,,
1100015,336,1918,23
1100023,336,9647,169
1100031,336,364,9
1100049,336,6125,79
1100056,336,693,13
...,...,...,...
5222005,336,339,5
5222054,336,287,7
5222203,336,70,2


In [8]:
# informações adicionais
df_x = pd.DataFrame({
    'atrib': ['municipios','populacao','dias', 'casos', 'mortes','data_inicio', 'data_termino'],
    'valor': [len(df_m), df_m['populacao'].sum(), len(df_t['dia']), df_c['casos'].sum(), 
              df_c['mortes'].sum(), df_t['data_coleta'].min(), df_t['data_coleta'].max()]})


df_r = pd.read_csv(path2 + 'lista_rel.csv')
df_x = df_x.append([{'atrib':'con_aer', 'valor':df_r['AER'].sum()},
                    {'atrib':'con_fer', 'valor':df_r['FER'].sum()},
                    {'atrib':'con_hid', 'valor':df_r['HID'].sum()},
                    {'atrib':'con_rod', 'valor':df_r['ROD'].sum()},
                    {'atrib':'con_fro', 'valor':df_r['FRO'].sum()}], ignore_index=True)

df_x.to_csv(path2 + 'atrib_extra.csv', index=False)
df_x

,atrib,valor
0,municipios,5570
1,populacao,211755692
2,dias,336
3,casos,8659640
4,mortes,213896
5,data_inicio,2020-02-25
6,data_termino,2021-01-25
7,con_aer,876
8,con_fer,1272
9,con_hid,1240
